In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install -U tfds-nightly

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output
import matplotlib.pyplot as plt

## Загрузка датасета Oxford-IIIT Pets

Датасет является стандартным датасетом для TensorFlow, однако как упомяналось выше возможно необходимо будет установить модуль tensorflow-datasets для python.

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Следующий код выполнит простую аугументацию данных посредством переворота изображений. В дополнение изображение будет нормализовано к 0 и 1. Пиксели сегментационной маски будут помечены {1, 2, 3}, но для удобства из данного цифрового ряда будет вычтено по 1 и в итоге получиться {0, 1, 2}

In [ ]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

In [ ]:
@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [ ]:
def load_image_test(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

Датасет уже содержит необходимые тестовый и тренеровочный сплиты, поэтому давайте использовать их.

In [ ]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 128
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)

In [ ]:
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

Давайте посмотрим на пример  изображения из датасета и соотвествующую ему маску из датасета.

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
for image, mask in train.take(1):
  sample_image, sample_mask = image, mask
display([sample_image, sample_mask])

## Определение модели (encoder VGG16)

Будем использовать модифицированный U-Net. В качестве энкодера будет использоваться предтренированный VGG16.
Декодером будет апсемпл блок уже имплементированный в TensorFlow examples [Pix2pix tutorial](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py).

In [ ]:
# ОСНОВНЫЕ НАБЛЮДЕНИЯ 
# Пример из keras tutorials segmentation очень долго тренируется (видимо потому что encoder не предобучен)
# надо его точность еще раз проверить

# ternausnet с ResNet152V2 энкодером за 20 эпох до 0.88. Такой же результат в домашке, где энкодер - VGG16
# учится быстрее, чем пример из keras



# Тип 1

In [ ]:
base_model = tf.keras.applications.ResNet152V2(include_top=False, weights='imagenet', input_shape=[128, 128, 3])

In [ ]:
#base_model.summary()

In [ ]:
# Как ternausnet но с ResNet152V2 энкодером (сам писал)

layer_names = [
    'input_10',   # 128x128 3
    'conv1_conv',   # 64x64 64
    'conv2_block3_1_conv',   # 32x32 64
    'conv3_block8_1_conv',   # 16x16 128
    'conv4_block36_1_conv',  # 8x8 256
    'post_relu', #  4x4 2048

]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
    pix2pix.upsample(32, 3),   # 64x64 -> 128x128
]

def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs
  skips = down_stack(x)
  x = tf.keras.layers.Conv2D(1024, 3, strides=(1, 1), padding='same', activation='relu')(skips[5])
  #x = tf.keras.layers.BatchNormalization(x)

  x = up_stack[0](x)
  x = tf.keras.layers.Concatenate()([x, skips[4]])
  x = tf.keras.layers.Conv2D(512, 3, strides=(1, 1), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(512, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.BatchNormalization(x)

  x = up_stack[1](x)
  x = tf.keras.layers.Concatenate()([x, skips[3]])
  x = tf.keras.layers.Conv2D(256, 3, strides=(1, 1), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(256, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.BatchNormalization(x)

  x = up_stack[2](x)
  x = tf.keras.layers.Concatenate()([x, skips[2]])
  x = tf.keras.layers.Conv2D(128, 3, strides=(1, 1), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(128, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.BatchNormalization(x)

  x = up_stack[3](x)
  x = tf.keras.layers.Concatenate()([x, skips[1]])
  x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.BatchNormalization(x)

  x = up_stack[4](x)
  x = tf.keras.layers.Concatenate()([x, skips[0]])
  #x = tf.keras.layers.Conv2D(3, 3, strides=(1, 1), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

  #x = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2, padding='same')(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
OUTPUT_CHANNELS = 3
model = unet_model(OUTPUT_CHANNELS)

# Тип 2

In [ ]:
# Пример из keras tutorials segmentation

def get_model(img_size, num_classes):
    inputs = tf.keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = tf.keras.layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = tf.keras.layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = tf.keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    previous_block_activation = x  # Set aside residual

    for filters in [256, 128, 64, 32]:
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.UpSampling2D(2)(x)

        # Project residual
        residual = tf.keras.layers.UpSampling2D(2)(previous_block_activation)
        residual = tf.keras.layers.Conv2D(filters, 1, padding="same")(residual)
        x = tf.keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = tf.keras.layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)
    #outputs = layers.Conv2D(3, 3, strides=(1, 1), padding='same', activation='relu')(x)
    # Define the model
    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
img_size = (128, 128)
num_classes = 3
model = get_model(img_size, num_classes)

# Тип 3

In [ ]:
# энкодер из tensorflow tutorials segmentation (mobile v2). с декодером экспериментировал, но результат 
# не особо изменился. Декодер такой же, но расписан не циклом как в туториале. Самый быстрый вариант

base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

In [ ]:
#base_model.summary()

In [ ]:
# Use the activations of these layers
layer_names = [
    'input_1',              # 128x128 3 (не используется, так как прироста точности не дает)
    'block_1_expand_relu',   # 64x64 96
    'block_3_expand_relu',   # 32x32 144
    'block_6_expand_relu',   # 16x16 192
    'block_13_expand_relu',  # 8x8 576
    'block_16_project',      # 4x4 320
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

In [ ]:
#base_model.summary()

In [ ]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
# взял код из tensorflow tutorials segmentation (mobile v2) и развернул для наглядности
# добавление одного сверточного слоя после конкатенации незначительно повысило точность (обучение замедлилось)
# добавление второго сверточного слоя после конкатенации значительно замедлило обучение
from tensorflow.keras import initializers

def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]
  #skips = reversed(skips[:-1])

  x = up_stack[0](x)
  x = tf.keras.layers.Concatenate()([x, skips[4]])
  #x = tf.keras.layers.Conv2D(760, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.Conv2D(512, 3, strides=(1, 1), padding='same', activation='relu')(x)

  x = up_stack[1](x)
  x = tf.keras.layers.Concatenate()([x, skips[3]])
  #x = tf.keras.layers.Conv2D(350, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.Conv2D(256, 3, strides=(1, 1), padding='same', activation='relu')(x)

  x = up_stack[2](x)
  x = tf.keras.layers.Concatenate()([x, skips[2]])
  #x = tf.keras.layers.Conv2D(200, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.Conv2D(128, 3, strides=(1, 1), padding='same', activation='relu')(x)

  x = up_stack[3](x)
  x = tf.keras.layers.Concatenate()([x, skips[1]])
  #x = tf.keras.layers.Conv2D(100, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='relu')(x)

  # This is the last layer of the model
  #last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same')  #64x64 -> 128x128

  
  last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same', activation='softmax')  #64x64 -> 128x128
  x = last(x)

  #x = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same', activation='relu')(x)
  #x = tf.keras.layers.Concatenate()([x, skips[0]])
  #x = tf.keras.layers.Conv2D(3, 3, strides=(1, 1), padding='same', activation='relu')(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
OUTPUT_CHANNELS = 3
model = unet_model(OUTPUT_CHANNELS)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

# Тип 4

EfficientNetB0

In [ ]:
! pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
# будет работать с элементами tensoflow.keras
import efficientnet.tfkeras as efn

# будет работать с элементами keras
#import efficientnet.keras as efn

In [ ]:
import tensorflow as tf

In [ ]:
model = efn.EfficientNetB0(weights='imagenet', input_shape=[128, 128, 3], include_top=False)  # or weights='noisy-student'

In [ ]:
model.trainable = False

In [ ]:
# 30.06.2020 лучше дергать скипы из слоев энкодера с меньшим количеством каналов (получилось
# быстрее и точнее), и при этом после каждой конкатенации применять конволюцию

In [ ]:
# Use the activations of these layers

# Берем слои с меньшим количеством каналов (из энкодера)
layer_names = [
    'stem_conv',   # 64x64 32
    'block2b_add',   # 32x32 24
    'block3b_add',   # 16x16 40
    'block5c_add',  # 8x8 112
    'top_activation',      # 4x4 1280 (top)
]

In [ ]:
# Берем слои с бОльшим количетсвом каналов (из энкодера)
layer_names = [
    'block2a_expand_conv',   # 64x64 96
    'block2b_expand_conv',   # 32x32 144
    'block3b_expand_conv',   # 16x16 240
    'block4b_expand_conv',  # 8x8 480
    'top_activation',      # 4x4 1280 (top)
]

In [ ]:
layers = [model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=model.input, outputs=layers)

down_stack.trainable = False

In [ ]:
#model.summary()

In [ ]:
#tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
#model.summary()

In [ ]:
# Соответствует слоям с меньшим количеством каналов 

up_stack = [
    pix2pix.upsample(224, 3),  # 4x4 -> 8x8
    pix2pix.upsample(80, 3),  # 8x8 -> 16x16
    pix2pix.upsample(48, 3),  # 16x16 -> 32x32
    pix2pix.upsample(32, 3),   # 32x32 -> 64x64
]

In [ ]:
# Соответствует слоям с бОльшим количеством каналов 

up_stack = [
    pix2pix.upsample(480, 3),  # 4x4 -> 8x8
    pix2pix.upsample(240, 3),  # 8x8 -> 16x16
    pix2pix.upsample(144, 3),  # 16x16 -> 32x32
    pix2pix.upsample(96, 3),   # 32x32 -> 64x64
]

In [ ]:
# Замутить хитрую инициализацию весов и swish

def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]

  x = tf.keras.layers.Conv2D(512, 3, strides=(1, 1), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(256, 3, strides=(1, 1), padding='same', activation='relu')(x)

  x = up_stack[0](x)
  x = tf.keras.layers.Concatenate()([x, skips[3]])
  x = tf.keras.layers.Conv2D(128, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='relu')(x)

  x = up_stack[1](x)
  x = tf.keras.layers.Concatenate()([x, skips[2]])
  x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.Conv2D(32, 3, strides=(1, 1), padding='same', activation='relu')(x)

  x = up_stack[2](x)
  x = tf.keras.layers.Concatenate()([x, skips[1]])
  x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.Conv2D(32, 3, strides=(1, 1), padding='same', activation='relu')(x)

  x = up_stack[3](x)
  x = tf.keras.layers.Concatenate()([x, skips[0]])
  x = tf.keras.layers.Conv2D(16, 3, strides=(1, 1), padding='same', activation='relu')(x)
  #x = tf.keras.layers.Conv2D(8, 3, strides=(1, 1), padding='same', activation='relu')(x)

  # This is the last layer of the model
  #last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same')  #64x64 -> 128x128

  
  last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same', activation='softmax')  #64x64 -> 128x128
  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
OUTPUT_CHANNELS = 3
model = unet_model(OUTPUT_CHANNELS)

## ТИП 5 - самый лучший вариант

EfficientNetB4

In [ ]:
! pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
import efficientnet.tfkeras as efn

In [ ]:
model = efn.EfficientNetB4(weights='imagenet', input_shape=[128, 128, 3], include_top=False)

In [ ]:
model.trainable = False

In [ ]:
layer_names = [
    'stem_conv',   # 64x64 48
    'block2b_add',   # 32x32 32
    'block3b_add',   # 16x16 56
    'block5c_add',  # 8x8 160
    'top_activation',      # 4x4 1792 (top)
]

In [ ]:
#model.summary()

In [ ]:
layers = [model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=model.input, outputs=layers)

down_stack.trainable = False

In [ ]:
# Замутить хитрую инициализацию весов и swish

def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]

  #x = tf.keras.layers.Conv2D(1024, 3, strides=(1, 1), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(512, 3, strides=(1, 1), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(256, 3, strides=(1, 1), padding='same', activation='relu')(x)

  x = up_stack[0](x)
  x = tf.keras.layers.Concatenate()([x, skips[3]])
  x = tf.keras.layers.Conv2D(128, 3, strides=(1, 1), padding='same', activation='relu')(x)


  x = up_stack[1](x)
  x = tf.keras.layers.Concatenate()([x, skips[2]])
  x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='relu')(x)


  x = up_stack[2](x)
  x = tf.keras.layers.Concatenate()([x, skips[1]])
  x = tf.keras.layers.Conv2D(64, 3, strides=(1, 1), padding='same', activation='relu')(x)


  x = up_stack[3](x)
  x = tf.keras.layers.Concatenate()([x, skips[0]])
  x = tf.keras.layers.Conv2D(32, 3, strides=(1, 1), padding='same', activation='relu')(x)
  x = tf.keras.layers.Conv2D(16, 3, strides=(1, 1), padding='same', activation='relu')(x)


  # This is the last layer of the model
  #last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same')  #64x64 -> 128x128

  
  last = tf.keras.layers.Conv2DTranspose(output_channels, 3, strides=2,padding='same', activation='softmax')  #64x64 -> 128x128
  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
OUTPUT_CHANNELS = 3
model = unet_model(OUTPUT_CHANNELS)

## Тренировка модели

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

Давайте попробуем сделать предсказание с помощью нашей модели до того как началось обучение.

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
show_predictions()

Давайте осуществлять мониторинг того как улучшается работа модели в процессе обучения. Для завершения этой задачи callback функция определена ниже.

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
EPOCHS = 50
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_dataset,
                          callbacks=[DisplayCallback()])

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

## Предсказание


Давайте сделаем несколько предсказаний. Для экономии времени использовалось небольшое количество эпох, но вы можете его увеличить для того чтобы модель давала более точные результаты.

In [ ]:
show_predictions(test_dataset, 3)